In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [59]:
filenames = ["result_msr_MSR_fit0_its100000_addprob1_time26_20_13_22.csv",
            "result_reproduce_fit0_its100000_time20_20_10_36.csv"
            "result_no_rec_fit0_its100000_time21_11_16_04.csv",
            "result_delete_state_fit0_its100000_addprob0.8_pref0_time26_21_38_24.csv",
            "result_reproduce_fit0_its100000_addprob1_pref1_time26_21_46_07.csv"]

filenames = [
    "result_msr_MSR_fit0_its100000_addprob1_time26_20_13_22.csv",
            "result_reproduce_fit0_its100000_time20_20_10_36.csv",
            "result_no_rec_fit0_its100000_time21_11_16_04.csv",
            "result_delete_state_fit0_its100000_addprob0.8_pref0_time26_21_38_24.csv",
            "result_reproduce_fit0_its100000_addprob1_pref1_time26_21_46_07.csv"
]

xtick_labels = [
    'with_subclass', 
    'basic_model',
    'no_recursion',
    'delete_statements',
    'caller_pref',
    'calle_pref',
    'no_pref'
]

In [27]:
data = pd.read_csv('commit_sizes/commits_' + filenames[1])
data

,sim,commit,size
0,0,0,0
1,0,1,5
2,0,2,1
3,0,3,1
4,0,4,1
5,0,5,1
6,0,6,1
7,0,7,14
8,0,8,1
9,0,9,1


In [104]:
def load_pickles(filenames):
    dicts = []
    for file_name in filenames:
        data = pd.read_csv('commit_sizes/' + filename)
        
        xmin, gamma, significances = fit_powerlaw(data)
        
from scipy.optimize import curve_fit
import random
from statsmodels.sandbox.stats.multicomp import multipletests


def func_powerlaw(x, a, c, c0):
    return c0 + x**a * c


def boostrapped_distr(data, sample_size, N):
    """
    Create bootstrapped distribution
    """
    gammas = []
    xmins = []
    for i in range(N):
        # Create synthetic data
        data_synt = random.choices(data['size'].values, k=sample_size)
        
        # Get fit results
        results = powerlaw.Fit(data_synt)
        gammas.append(results.power_law.alpha)
        xmins.append(results.power_law.xmin)    
        
    gammas = sorted(gammas)
    xmins = sorted(xmins)
    return gammas, xmins


def find_p_value(value, distr):
    """
    Returns proportion values smaller than value in the distribution"""
    s = 0
    for x in distr:
        if x >= value:
            break
        s += 1
        
    p_value = s/len(distr)
    if p_value <.5:
        return p_value
    else:
        return 1-p_value
    

def fit_powerlaw(data, N_boot):
    # Mean sample size
    sample_size = int(np.array([len(data[data['sim']==sim]) for sim in data['sim'].unique()]).mean())

    # Create bootstrapped distribution for gamma and for MSE
    gamma_distr, xmins_distr = boostrapped_distr(data, sample_size, N_boot)
    MSE_distr = xmins_distr
    
    # Get gamma and MSE for the real simulations
    gammas = []
    xmins = []
    MSEs = []
    
    for sim in data['sim'].unique():        
        # Get gamma and MSE
        results = powerlaw.Fit(data['size'][data['sim']==sim].values)
        gammas.append(results.power_law.alpha)
        xmins.append(results.power_law.xmin)

    return gammas, gamma_distr, MSEs, MSE_distr
    
    
    
def get_data(filenames, labels):
    distrs = {}
    names = []
    for i in range(len(filenames)):
        data = pd.read_csv('commit_sizes/commits_' + filenames[i])
        
        gammas, gamma_distr, MSEs, MSE_distr = fit_powerlaw(data, 60)
        print(labels[i])
        distrs[labels[i]+'gamma'] = gammas
#         distrs[labels[i]+'gamma_distr'] = gamma_distr
#         distrs[labels[i]+'MSE'] = MSEs
#         distrs[labels[i]+'MSE_distr'] = MSE_distr
        
        names += [labels[i]+'gamma']
#                   , labels[i]+'gamma_distr']
#                   , labels[i]+'MSE', labels[i]+'MSE_distr']
    return distrs, names
        
# fit_powerlaw(data, 10**1)  

def get_means(results):
    means = {}
    for result in results:
        means[result] = np.array(results[result]).mean()
    return means


import scipy.stats as stats
def ANOVA(filenames, labels):
    results, names = get_data(filenames, xtick_labels)
    print(stats.f_oneway(*[results[name] for name in names]))
    means = get_means(results)
    print(means)
    
    
# def t_tests()

In [105]:
ANOVA(filenames, xtick_labels)

Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than

Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best

with_subclass


Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best

Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best

...

delete_statements


Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best

Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than

caller_pref
F_onewayResult(statistic=0.845270410910067, pvalue=0.4973992687471074)
{'with_subclassgamma': 1.6621440582015758, 'basic_modelgamma': 1.660549077648692, 'no_recursiongamma': 1.6583216337128126, 'delete_statementsgamma': 1.6652325956001601, 'caller_prefgamma': 1.6783351334725038}



Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit
Values less than or equal to 0 in data. Throwing out 0 or negative values
Calculating best minimal value for power law fit


In [ ]:
results

In [23]:
import powerlaw
data = np.array([1.7, 3.2, 5]) # data can be list or numpy array
results = powerlaw.Fit(data)
# print(results.powerlaw)
print(results.power_law.alpha)
print(results.power_law.xmin)
print(results.power_law.D)

power = [range(1,4)]
mse = ((data - power)**2).mean(axis=ax)

attrs = vars(results)
print(', '.join("%s: %s" % item for item in attrs.items()))

R, p = results.distribution_compare('power_law', 'lognormal')

2.753020228765184
1.7
0.33672019364008626
data_original: [1.7 3.2 5. ], data: [1.7 3.2 5. ], discrete: False, fit_method: Likelihood, estimate_discrete: True, discrete_approximation: round, sigma_threshold: None, parameter_range: None, given_xmin: None, given_xmax: None, xmin: 1.7, xmax: None, xmin_distance: D, fixed_xmax: False, fitting_cdf_bins: [1.7 3.2 5. ], fitting_cdf: [0.         0.33333333 0.66666667], fixed_xmin: False, Ds: [0.33672019 0.36466472], alphas: [2.75302023 5.48142012], sigmas: [1.0121067  3.16884255], in_ranges: [ True  True], xmins: [1.7 3.2], noise_flag: False, D: 0.33672019364008626, alpha: 2.753020228765184, sigma: 1.0121067009724383, n: 3.0, n_tail: 3.0, supported_distributions: {'power_law': <class 'powerlaw.Power_Law'>, 'lognormal': <class 'powerlaw.Lognormal'>, 'exponential': <class 'powerlaw.Exponential'>, 'truncated_power_law': <class 'powerlaw.Truncated_Power_Law'>, 'stretched_exponential': <class 'powerlaw.Stretched_Exponential'>, 'lognormal_positive': 

Calculating best minimal value for power law fit
